### This script aims to explore the full dataset build in "EDA" and find feature importance

In [65]:
## Packages and data
# Load packages
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
import statsmodels.formula.api as sm1
import statsmodels.discrete.discrete_model as sm2
import pymc3 as pm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline


# Load data
df = pd.read_pickle('clean_data/df.pkl')

In [66]:
# Prepare data for modeling
X = df.iloc[:,2:].drop("MONTH",1)
y1 = df['AQI']
y2 = df['HEALTHY']

f = ("AQI ~ WND + PRCP + TMP + TRAF + MANU + MONTH") # Define formula

print(df.columns)
print(df.shape)

Index(['AQI', 'HEALTHY', 'WND', 'PRCP', 'TMP', 'TRAF', 'MANU', 'MONTH',
       'MONTH1', 'MONTH2', 'MONTH3', 'MONTH4', 'MONTH5', 'MONTH6', 'MONTH7',
       'MONTH8', 'MONTH9', 'MONTH10', 'MONTH11'],
      dtype='object')
(2832, 19)


In [67]:
# Fit linear regression
fit_lin = sm1.ols(formula=f, data=df).fit()
display(fit_lin.summary()) # Multicolinearity issue

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    AQI   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     93.28
Date:                Tue, 27 Jul 2021   Prob (F-statistic):          3.62e-107
Time:                        07:15:12   Log-Likelihood:                -12422.
No. Observations:                2832   AIC:                         2.486e+04
Df Residuals:                    2825   BIC:                         2.490e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     51.7387      2.814     18.389      0.000      46.222      57.256
WND           -0.4808      0.027    -17.974      0.000      -0.533      -0.428
PRCP          -0.0744      0.012     -6.223      0.000      -0.098      -0.051
TMP           -0.0123      0.004     -3.198      0.001      -0.020      -0.005
TRAF           0.0001      0.000      0.334      0.738      -0.000       0.001
MANU          -0.0059      0.001     -4.221      0.000      -0.009      -0.003
MONTH          0.4449      0.109      4.067      0.000       0.230       0.659
==============================================================================
Omnibus:                     1153.951   Durbin-Watson:                   0.533
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5313.096
Skew:                           1.942   Prob(JB):                         0.00
Kurtosis:                       8.472   Cond. No.                     5.52e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.52e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [68]:
# Fit logistic regression
fit_log = sm2.Logit(y2, X).fit()
display(fit_log.summary()) # Multicolinearity issue

         Current function value: 0.046097
         Iterations: 35


/home/blakejhamm/.conda/envs/airq_eda/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                HEALTHY   No. Observations:                 2832
Model:                          Logit   Df Residuals:                     2816
Method:                           MLE   Df Model:                           15
Date:                Tue, 27 Jul 2021   Pseudo R-squ.:                  0.4807
Time:                        07:15:15   Log-Likelihood:                -130.55
converged:                      False   LL-Null:                       -251.39
Covariance Type:            nonrobust   LLR p-value:                 6.364e-43
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
WND            0.2253      0.035      6.495      0.000       0.157       0.293
PRCP          -0.0051      0.012     -0.415      0.678      -0.029       0.019
TMP            0.0161      0.004      3.758      0.000       0.008       0.025
TRAF         -7.1e-05      0.000     -0.708      0.479      -0.000       0.000
MANU          -0.0005      0.001     -0.814      0.416      -0.002       0.001
MONTH1        -0.7560      0.383     -1.974      0.048      -1.507      -0.005
MONTH2        -1.7643      0.542     -3.256      0.001      -2.826      -0.702
MONTH3        17.3774   6655.765      0.003      0.998    -1.3e+04    1.31e+04
MONTH4        14.4157   2782.217      0.005      0.996   -5438.630    5467.461
MONTH5        20.5173   8.01e+04      0.000      1.000   -1.57e+05    1.57e+05
MONTH6        16.0399   1.94e+04      0.001      0.999    -3.8e+04     3.8e+04
MONTH7        14.2307   1.15e+04      0.001      0.999   -2.25e+04    2.25e+04
MONTH8        -5.0158      1.497     -3.351      0.001      -7.950      -2.082
MONTH9        -2.5524      1.381     -1.848      0.065      -5.260       0.155
MONTH10       14.6272   1813.381      0.008      0.994   -3539.534    3568.788
MONTH11       19.4629   8743.761      0.002      0.998   -1.71e+04    1.72e+04
==============================================================================

Possibly complete quasi-separation: A fraction 0.66 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [69]:
# Fit Bayesian Ridge Regression
reg = linear_model.BayesianRidge()
fit_bays = reg.fit(X, y1)
score = fit_bays.score(X, y1)
print("Model score (R-squared): %.2f" % score)
print(fit_bays.coef_.round(3)) # Similar coefficients to linear model

Model score (R-squared): 0.32
[-4.0300e-01 -6.4000e-02  1.2000e-02  1.0000e-03 -1.0000e-03  6.6580e+00
 -1.4876e+01 -2.3805e+01 -2.3122e+01 -2.3627e+01 -1.8711e+01 -1.1900e+01
 -9.1270e+00 -1.3946e+01 -2.0144e+01 -1.2529e+01]


In [70]:
# Fit random forest regression
regr = RandomForestRegressor(max_depth=2, random_state=0)
fit_for = regr.fit(X, y1)
score = fit_for.score(X, y1)
print("Model score (R-squared): %.2f" % score)
print(fit_for.feature_importances_.round(3)) # Only two relevant features...

Model score (R-squared): 0.42
[0.701 0.    0.299 0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.   ]


In [77]:
# Try PCR because multicolinearity
pcr = make_pipeline(StandardScaler(), PCA(), linear_model.LinearRegression())
fit_pcr = pcr.fit(X, y1)
pca = pcr.named_steps['pca']  # retrieve the PCA step of the pipeline
score = fit_pcr.score(X, y1)
print("Model score (R-squared): %.2f" % score)
#print(fit_pcr.coef_.round(3)) # Only two relevant features...

Model score (R-squared): 0.32
